- 참고 링크 : https://gwkim.tistory.com/m/2

In [2]:
# 라이브러리 import 
from bs4 import BeautifulSoup
import requests
import time 
import random 
from tqdm import tqdm 
from collections import defaultdict 
import json
import pandas as pd

In [ ]:
# 파라미터 및 관련 변수 설정
# 원하는 주소의 위 , 경도 주소가 필요!

#url = 'https://www.yogiyo.co.kr/api/v1/restaurants/1115073/menu/'
url = 'https://www.yogiyo.co.kr/api/v1/restaurants-geo/'
api_key = 'fe5183cc3dea12bd0ce299cf110a75a2'

## 특정 위경도의 "치킨" 카테고리를 조회 
parameters = {"category" : "한식" ,
#              "items" : 20 ,
            "lat" : 37.5565050755347,
            "lng" : 126.939656244325,
#             "order" : 'review_count', ## 정렬 기준
            "page" : 0
             }
headers = {'X-ApiSecret':api_key,'X-ApiKey':"iphoneap"}
res = requests.get(url,params=parameters, headers=headers)
info = json.loads(res.content)

In [24]:
with open('../프로젝트_최종/요기요.json', 'w') as outfile:
    json.dump(info, outfile, indent=4)

In [3]:
##  필요한 항목 
key_list =["id" , "name" , "address" , "estimated_delivery_time" ,"min_order_amount"]
## dataframe 만들기 
df_list =[]
for idx in range(len(info["restaurants"])):
    info_list=[]
    for key in key_list:
        info_list.append(info["restaurants"][idx][key])
    df_list.append(info_list)
    
## dataframe으로 표출하기 
df = pd.DataFrame(df_list , columns = key_list)

print(df.shape)  ##(280, 5)
print(df.info()) 

(100, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       100 non-null    int64 
 1   name                     100 non-null    object
 2   address                  100 non-null    object
 3   estimated_delivery_time  100 non-null    object
 4   min_order_amount         100 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 4.0+ KB
None


In [4]:
df.head(2)

,id,name,address,estimated_delivery_time,min_order_amount
0,1138033,엽전식당-연남점,서울특별시 마포구 연남동 227-34 1층,65~75분,9000
1,377422,백순대본가새맛-연대점,서울특별시 서대문구 창천동 52-149 1층,45~55분,5000


# 민규님 코드

In [ ]:
from bs4 import BeautifulSoup
import requests
import time 
import random 
from tqdm import tqdm 
from collections import defaultdict 
import json
import pandas as pd

In [ ]:
# 요기요 상품 카테고리 페이지 크롤링
def yogiyo_page_crawling(geo_url, api_key, parameters, headers):
    res = requests.get(geo_url, params=parameters, headers=headers)
    page = json.loads(res.content)
    return page

# id, name, address 파라미터로 간단한 데이터프레임화
def yogiyo_dataframe(page):
    key_list =["id", "name", "address"]
    df_list =[]
    for idx in range(len(page["restaurants"])):
        info_list=[]
        for key in key_list:
            info_list.append(page["restaurants"][idx][key])
        df_list.append(info_list)
    df = pd.DataFrame(df_list , columns = key_list)
    return df

# 위에서 찾은 id로 점포 상세(메뉴 및 가격)페이지 다시 크롤링
def yogiyo_product_crawling(df, product_url, api_key, parameters, headers):
    df.id = df.id.astype(str)
    pageid = list(df.id.values)
    all_info = []
    # 상위 가게 3개만 뽑아오기
    for num in range(3):
        detailed_url = product_url + pageid[num] + '/menu/'        
        res = requests.get(detailed_url,params=parameters, headers=headers)
        info = json.loads(res.content)
        all_info.append(info)
    return all_info


# 요기요 점포 메뉴 , 가격 데이터프레임화
def yogiyo_menu_price(page):
    cnt = 0
    menu_df = pd.DataFrame(columns=['idx', 'menu_idx,', 'menu_name', 'price'], index = [0])
    for h in range(len(page)):
        for i in range(len(page[h])):
            for j in range(len(page[h][i].get('items'))):
                idx = h
                menu_idx = i
                menu_name = page[h][i].get('items')[j].get('name'), 
                price = page[h][i].get('items')[j].get('price')
                menu_df.loc[cnt] = [idx, menu_idx, menu_name, price]
                cnt += 1
    return menu_df

In [ ]:
# 실행 코드

category_list = ['고기구이', '피자양식', '야식', '치킨', '테이크아웃', 'SNS맛집', '신규맛집', '분식', '찜탕', '한식', '샌드위치', '샐러드', '카페디저트', 
                 '아시안', '버거', '족발보쌈', '회초밥', '도시락죽', '중식', '일식돈까스', '예약픽업']
spot = pd.read_excel('시군구위도경도.xlsx')[0:25]
lat_list = list(spot.y.values)
lng_list = list(spot.x.values)

geo_url = 'https://www.yogiyo.co.kr/api/v1/restaurants-geo/'
product_url = 'https://www.yogiyo.co.kr/api/v1/restaurants/'
api_key = 'fe5183cc3dea12bd0ce299cf110a75a2'

parameters = {"category" : '피자양식',
							#"items" : 20 ,
              "lat" : lat_list[0],
              "lng" : lng_list[0],
	            #"order" : 'review_count', ## 정렬 기준
              "page" : 0
							}
headers = {'X-ApiSecret':api_key,'X-ApiKey':"iphoneap"}

yogiyo_page = yogiyo_page_crawling(geo_url, api_key, parameters, headers)
yogiyo_df = yogiyo_dataframe(yogiyo_page)
yogiyo_product_page = yogiyo_product_crawling(yogiyo_df, product_url, api_key, parameters, headers)
yogiyo_menu = yogiyo_menu_price(yogiyo_product_page)

# 해야할것 

1. Category_list 따라 반복문 설정, 
2. yogiyo menu[’menu_name’] 텍스트 전처리, 
3. yogiyo_menu[’idx’]  가게 이름명으로 바꿔주기